<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Clustering" data-toc-modified-id="Clustering-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Clustering</a></span><ul class="toc-item"><li><span><a href="#K-means-clustering" data-toc-modified-id="K-means-clustering-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>K-means clustering</a></span><ul class="toc-item"><li><span><a href="#Cluster-ads-users" data-toc-modified-id="Cluster-ads-users-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Cluster ads users</a></span></li></ul></li><li><span><a href="#Clustering-New-York-City-collisions-data" data-toc-modified-id="Clustering-New-York-City-collisions-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Clustering New York City collisions data</a></span><ul class="toc-item"><li><span><a href="#Taking-a-subset-of-the-data-points-and-features" data-toc-modified-id="Taking-a-subset-of-the-data-points-and-features-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Taking a subset of the data points and features</a></span></li><li><span><a href="#Standardizing-the-data" data-toc-modified-id="Standardizing-the-data-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Standardizing the data</a></span></li><li><span><a href="#Visualize-the-data" data-toc-modified-id="Visualize-the-data-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Visualize the data</a></span></li>

# Clustering


In [2]:
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets as skdat
import scipy
from pandas.io import gbq

## K-means clustering


### Cluster ads users

Given the above pseudocode for K-means, complete the implementation below.

Guidance: the answer uses a main *while* loop (one for each outer iteration of K-means) and two *for* loops (one stepping through the data and another stepping through the clusters). 

In [3]:
q = """

SELECT
  user_id,
  SUM(IF(is_impression,1,0)) AS imp,
  SUM(num_streams ) AS stream
FROM
  `ad-veritas.oculus_impressions_base.oculus_impressions_base_2018*` a
JOIN
  `gro-analytics.custom_segs.user_demoPlatform_2018*` b
USING
  (user_id)
WHERE
  country = 'US'
AND product_cat = 'free'
AND a._table_suffix BETWEEN '0515'AND '0614'
AND b._table_suffix BETWEEN '0515'AND '0614'
AND RAND() <= 0.000001
GROUP BY
  1
"""

df_user_ads = gbq.read_gbq(q, dialect='standard', project_id='analytics-mafia')

In [4]:
df_user_ads['imp_normalized']

KeyError: 'imp_normalized'

In [ ]:
def kmeans(X, K, eps=1e-5, max_iterations=200):
    """
        arguments:
            X: a (N,D) numpy array of observed data
            K: integer indicating number of clusters
            eps (optional): real threshold for change in mu for deciding when to stop
            max_iterations (optional): max num of iterations, regardless of eps threshold
        returns:
            mu: a (K,D) numpy array of cluster means after k-means converged
            R: a (N,K) numpy array of binary cluster assignments
    """
    
    #todo: put your code here
    (N,D) = X.shape
    np.random.seed(100)
    rand_n = np.random.choice(range(N),size=K)
    mu = X[rand_n,:]
    prev_mu = np.inf #always do the first iteration
    R = np.zeros((N,K)) #this will be written over before first read
    it = 0
    while np.abs(mu-prev_mu).sum() >= eps and it<max_iterations:
        #store previous value of mu:
        prev_mu = mu.copy()

        #A. update cluster assignments according to nearest cluster
        for nrow in range(N):
            dist = np.array([np.linalg.norm(X[nrow,:] - mu[k,:]) for k in range(K)])
            R[nrow,:] = 0
            R[nrow,dist.argmin()] = 1

        #B. calculate cluster sizes
        cluster_size = R.sum(axis = 0) + eps

        #C. update cluster centers
        mu = np.dot(R.T, X) / cluster_size[:,np.newaxis]
        
        it += 1 #increment iteration
    
    return mu, R

In [ ]:
X = df_user_ads[['imp','stream']]
X.imp = X.imp/1000
X = X.as_matrix(columns=['imp','stream'])
X

In [ ]:

N,D,K = df_user_ads.shape[0], 2, 4

mu, R = kmeans(X,K)
print('cluster centers',mu)
print('cluster assignments',R.argmax(axis=1))

In [ ]:
#code for plotting data-cluster assignments using a different colour for each cluster:
#plt.ioff()
def plot_clusters_2D(X,R,mu,separate=False,titles=None):
    colors = ['r', 'g', 'b', 'y']
    A = R.argmax(axis=1) #calculate most likely cluster assignment
    for k in range(K):
        nsk, = np.where(A==k) #select data points based on assignment
        c = colors[k % len(colors)] #choose colour from list
        plt.scatter(X[nsk,0], X[nsk,1], marker='x', color=c)
        plt.scatter(mu[k,0],mu[k,1],color=c)
        if titles is not None: plt.title(titles[k])
        if separate and k<K-1: plt.figure()

In [ ]:
#plot the clusters for the toy data:
plot_clusters_2D(X,R,mu)

In [ ]:
print(R.shape, X.shape)

In [ ]:
X[R[:,0]==1,0]

## Clustering New York City collisions data

To end this lab session, we will apply the clustering methods you just implemented to a subset of the New York City collisions data set. The data set is the location of traffic collisions in New York City between June 1st 2016 and June 8th 2016. The data was obtained from https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95/data

We will use the `Pandas` library to simply load a CSV (comma-separated values) file of the data and display a summary (the first and last several rows) of the whole data file.

In [ ]:
import pandas as pd
data_path = './data/nyc_collisions_01june_08june_2016.csv'
collisions_table = pd.read_csv(data_path)
collisions_table #browse data in a table format

### Taking a subset of the data points and features
You will notice that there are multiple columns per collision and that not every collision has a related location. A more complex model may be able to incorporate this extra information but for now let's focus on just the locations of collisions and time of day. 

We filter the data: removing columns (features) and rows (collisions) so that we end up with a `numpy` array `X` of collisions with valid latitutes and longitudes.

In [ ]:
loc_collisions_table = collisions_table[np.isfinite(collisions_table['LATITUDE'])] #remove rows with NaNs
loc_collisions_table['TIME_HOUR'] = loc_collisions_table.TIME.apply(lambda x: float(x.split(':')[0]) + float(x.split(':')[1])/60.)
Xcol = loc_collisions_table.as_matrix(columns=['LONGITUDE','LATITUDE','TIME_HOUR'])
Xcol #display the data as a N-by-4 numpy array

### Standardizing the data
It is good practice to standardize the data, that is, to transform the data such that it has zero mean and unit standard deviation. This helps with hyperparameter selection and parameter exploration. But sometimes it's useful to work in the original space, so be sure to save the transformation variables for later.

In [ ]:
tr_mn = Xcol.mean(axis=0)
tr_sd = (Xcol-tr_mn).std(axis=0)
Xcol1 = (Xcol-tr_mn)/tr_sd
#X = (X1-tr_mn)/tr_var
print('transformation variables:\n mean=',tr_mn,'std=',tr_sd)
print('X=\n',Xcol1)
#check:
print('check transformed X properties:\n mean=',Xcol1.mean(axis=0),'std=',Xcol1.std(axis=0))

### Visualize the data
Now we are going to make a simple visualization of the data. Since `X` consists of 2-dimensional points we use a 2-dimensional scatter.

In [ ]:
import seaborn as sns
sns.set(color_codes=True)
ax = sns.lmplot("LONGITUDE","LATITUDE", data=loc_collisions_table, fit_reg=False)


 ```tr_mn``` and ```tr_sd``` is to 'undo' the effect of normalization so that we can have more interpretable plots

In [ ]:
K = 3 #choose num clusters
mu3, R3 = kmeans(Xcol1,K) #apply kmeans
print('cluster centres',mu3) #print centres
#create title for each plot including cluster average time:
titles = ['%i data points, average time of day = %.1f hours' % (R3.sum(axis=0)[k], mu3[k,2]*tr_sd[2]+tr_mn[2]) for k in range(K)]
plot_clusters_2D(Xcol1*tr_sd+tr_mn,R3,
                 mu3*tr_sd+tr_mn,
                 separate=True,titles=titles)